In [10]:
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

# 定义设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取CSV文件
df = pd.read_csv("最终的记录.csv", encoding='utf-8', encoding_errors='ignore')

# 重命名列（确保与之前一致）
df.columns = [
    '商品名称', '一级种类', '二级种类', '图片地址', 'sku', '文本描述', '折扣率',
    '折扣价', '价格', '星级', '销量', '收入', '评论', 'cc-1', 'cc-2',
    'cc-3', 'DRC-1', 'DRC-2', 'DRC-3', 'RCV-1', 'RCV-2',
    'RCV-3', 'RSV-1', 'RSV-2', 'RSV-3'
]


# 转换数据类型
numeric_columns = ['星级', '折扣率', '折扣价', '价格', '销量', '收入']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 重新计算收入（如果需要）
df['收入'] = df['折扣价'] * df['销量']

# 数值特征
numerical_features = ['星级', '折扣价', '折扣率', '价格']

# 标准化数值特征
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [13]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 假设 df 是您已经加载的 DataFrame，并包含以下列：
# ['星级', '折扣价', '折扣率', '价格', '收入']

# 提取特征和目标变量
X = df[['折扣价', '折扣率', '价格']]
y = df['收入']

# 定义随机种子列表
random_states = [42, 23, 15, 34, 18, 32, 47, 27, 8, 52]

# 初始化结果存储结构
results = {
    'random_state': [],
    'MAE': [],
    'RMSE': [],
    'R²': [],
    'Feature Importances': [],
    'N Estimators': [],
    'Max Depth': []
}

# 初始化变量以跟踪最佳结果
best_r2 = -np.inf
best_params = {}

# 遍历每个随机种子
for state in random_states:
    print(f"\n=== random_state = {state} ===")
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=state
    )
    
    # 创建随机森林回归模型
    rf_model = RandomForestRegressor(n_estimators=100, random_state=state, max_depth=None)
    
    # 训练模型
    rf_model.fit(X_train, y_train)
    
    # 进行预测
    y_pred_rf = rf_model.predict(X_test)
    
    # 计算评估指标
    mae = mean_absolute_error(y_test, y_pred_rf)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
    r2 = r2_score(y_test, y_pred_rf)
    
    print(f"Random Forest MAE: {mae:.4f}")
    print(f"Random Forest RMSE: {rmse:.4f}")
    print(f"Random Forest R²: {r2:.4f}")
    print(f"Feature Importances: {rf_model.feature_importances_}")
    print(f"Number of Estimators: {rf_model.n_estimators}")
    print(f"Max Depth: {rf_model.max_depth}")
    
    # 记录结果
    results['random_state'].append(state)
    results['MAE'].append(mae)
    results['RMSE'].append(rmse)
    results['R²'].append(r2)
    results['Feature Importances'].append(rf_model.feature_importances_.tolist())
    results['N Estimators'].append(rf_model.n_estimators)
    results['Max Depth'].append(rf_model.max_depth)
    
    # 检查是否为最佳结果
    if r2 > best_r2:
        best_r2 = r2
        best_params = {
            'random_state': state,
            'MAE': mae,
            'RMSE': rmse,
            'R²': r2,
            'Feature Importances': rf_model.feature_importances_.tolist(),
            'N Estimators': rf_model.n_estimators,
            'Max Depth': rf_model.max_depth
        }

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results)

# 展示汇总结果
print("\n=== 所有实验的结果汇总 ===")
print(results_df)

# 可选：保存实验结果到 CSV 文件
results_df.to_csv("random_forest_results_tuned.csv", index=False, encoding='utf-8-sig')
print("\n实验结果已保存到 'random_forest_results_tuned.csv'")



=== random_state = 42 ===
Random Forest MAE: 1372.7492
Random Forest RMSE: 2456.4349
Random Forest R²: -0.2715
Feature Importances: [0.38964947 0.29063472 0.31971581]
Number of Estimators: 100
Max Depth: None

=== random_state = 23 ===
Random Forest MAE: 1695.6355
Random Forest RMSE: 6180.3453
Random Forest R²: 0.0253
Feature Importances: [0.38446749 0.30085824 0.31467427]
Number of Estimators: 100
Max Depth: None

=== random_state = 15 ===
Random Forest MAE: 1340.6474
Random Forest RMSE: 2638.7389
Random Forest R²: -0.0483
Feature Importances: [0.36740073 0.28491865 0.34768062]
Number of Estimators: 100
Max Depth: None

=== random_state = 34 ===
Random Forest MAE: 1649.5532
Random Forest RMSE: 5573.4806
Random Forest R²: -0.0666
Feature Importances: [0.35517371 0.2457159  0.39911039]
Number of Estimators: 100
Max Depth: None

=== random_state = 18 ===
Random Forest MAE: 1604.4614
Random Forest RMSE: 5174.4288
Random Forest R²: -0.0034
Feature Importances: [0.3855704 0.2214478 0.39298

In [3]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 假设 df 是您已经加载的 DataFrame，并包含以下列：
# ['星级', '折扣价', '折扣率', '价格', '收入']

# 提取特征和目标变量
X = df[['折扣价', '折扣率', '价格']]
y = df['收入']

# 定义随机种子列表
random_states = [42, 23, 15, 34, 18, 32, 47, 27, 8, 52]

# 定义支持向量机的 C 和 epsilon 参数列表
C_values = [0.1, 1.0, 10.0]
epsilon_values = [0.01, 0.1, 0.2]

# 初始化结果存储结构
results = {
    'random_state': [],
    'C': [],
    'epsilon': [],
    'MAE': [],
    'RMSE': [],
    'R²': [],
    'Support Vectors': [],
}

# 初始化变量以跟踪最佳结果
best_r2 = -np.inf
best_params = {}

# 遍历每个 C 和 epsilon 的组合
for C in C_values:
    for epsilon in epsilon_values:
        for state in random_states:
            print(f"\n=== C = {C}, epsilon = {epsilon}, random_state = {state} ===")
            
            # 划分训练集和测试集
            X_train, X_test, y_train, y_test = train_test_split(
                X,
                y,
                test_size=0.2,
                random_state=state
            )
            
            # 创建支持向量机回归模型
            svr_model = SVR(C=C, epsilon=epsilon, kernel='rbf')
            
            # 训练模型
            svr_model.fit(X_train, y_train)
            
            # 进行预测
            y_pred_svr = svr_model.predict(X_test)
            
            # 计算评估指标
            mae = mean_absolute_error(y_test, y_pred_svr)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred_svr))
            r2 = r2_score(y_test, y_pred_svr)
            
            print(f"SVM MAE: {mae:.4f}")
            print(f"SVM RMSE: {rmse:.4f}")
            print(f"SVM R²: {r2:.4f}")
            print(f"Support Vectors: {svr_model.support_}")
            
            # 记录结果
            results['random_state'].append(state)
            results['C'].append(C)
            results['epsilon'].append(epsilon)
            results['MAE'].append(mae)
            results['RMSE'].append(rmse)
            results['R²'].append(r2)
            results['Support Vectors'].append(len(svr_model.support_))
            
            # 检查是否为最佳结果
            if r2 > best_r2:
                best_r2 = r2
                best_params = {
                    'random_state': state,
                    'C': C,
                    'epsilon': epsilon,
                    'MAE': mae,
                    'RMSE': rmse,
                    'R²': r2,
                    'Support Vectors': len(svr_model.support_)
                }

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results)

# 展示汇总结果
print("\n=== 所有实验的结果汇总 ===")
print(results_df)

# 可选：保存实验结果到 CSV 文件
results_df.to_csv("svm_regression_results_tuned.csv", index=False, encoding='utf-8-sig')
print("\n实验结果已保存到 'svm_regression_results_tuned.csv'")



=== C = 0.1, epsilon = 0.01, random_state = 42 ===
SVM MAE: 1199.8151
SVM RMSE: 2301.7622
SVM R²: -0.1164
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_state = 23 ===
SVM MAE: 1563.7660
SVM RMSE: 6369.5625
SVM R²: -0.0353
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_state = 15 ===
SVM MAE: 1279.1260
SVM RMSE: 2732.6591
SVM R²: -0.1243
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_state = 34 ===
SVM MAE: 1629.8333
SVM RMSE: 5537.3977
SVM R²: -0.0528
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_state = 18 ===
SVM MAE: 1597.0593
SVM RMSE: 5314.5973
SVM R²: -0.0585
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_state = 32 ===
SVM MAE: 1231.3257
SVM RMSE: 3581.4899
SVM R²: -0.0474
Support Vectors: [   0    1    2 ... 1752 1753 1754]

=== C = 0.1, epsilon = 0.01, random_sta